In [1]:
import cirq
import numpy as np

In [2]:
#Create qudit hadamard class

class QuditHadamard(cirq.SingleQubitGate):
    
    def __init__(self, dim):
        self.dim = dim
        super(QuditHadamard, self)
        
    def _qid_shape_(self):
        return (self.dim,)

    def _unitary_(self):
        unitary = np.zeros((self.dim, self.dim), dtype = np.complex_)
        w = np.exp(2 * np.pi * 1j / self.dim)
        
        for i in range(self.dim):
            for j in range(self.dim):
                unitary[i, j] = w ** (i * j) / np.sqrt(self.dim)
        
        return unitary

    def _circuit_diagram_info_(self, args):
        return '[H+]'


In [3]:
#Create qudit inverse hadamard class

class QuditInverseHadamard(cirq.SingleQubitGate):
    
    def __init__(self, dim):
        self.dim = dim
        super(QuditInverseHadamard, self)
        
    def _qid_shape_(self):
        return (self.dim,)

    def _unitary_(self):
        unitary = np.zeros((self.dim, self.dim), dtype = np.complex_)
        w = np.exp(2 * np.pi * 1j / self.dim)
        
        #For inverse hadamard, H^-1[i][j] = H[i][dim - j] for j > 0
        for i in range(self.dim):
            for j in range(self.dim):
                unitary[i, j] = w ** (i * (self.dim - j)) / np.sqrt(self.dim)
        
        return unitary

    def _circuit_diagram_info_(self, args):
        return '[H-]'


In [4]:
#Create qudit cnot gate

class QuditCnot(cirq.Gate):
    def __init__(self, dim):
        self.dim = dim
        super(QuditCnot, self)
    
    def _num_qubits_self(self):
        return 2
    
    def _qid_shape_(self):
        return (self.dim, self.dim)

    def _unitary_(self):
        unitary = np.zeros((self.dim**2, self.dim**2))
        
        for i in range(self.dim):
            for j in range(self.dim):
                start = self.dim * i + j    #starting index of permutation
                end = self.dim * i + (i + j) % self.dim    #Ending index of permutation
                unitary[end, start] = 1
                
        return unitary

    def _circuit_diagram_info_(self, args):
        return 'O', 'X'

In [5]:
#Create qudit Inverse cnot gate

class QuditInverseCnot(cirq.Gate):
    def __init__(self, dim):
        self.dim = dim
        super(QuditInverseCnot, self)
    
    def _num_qubits_self(self):
        return 2
    
    def _qid_shape_(self):
        return (self.dim, self.dim)

    def _unitary_(self):
        unitary = np.zeros((self.dim**2, self.dim**2))
        
        for i in range(self.dim):
            for j in range(self.dim):
                start = self.dim * i + (i + j) % self.dim    #starting index of permutation
                end = self.dim * i + j    #Ending index of permutation
                unitary[end, start] = 1
                
        return unitary

    def _circuit_diagram_info_(self, args):
        return 'O', 'Y'

In [6]:
#Qudit raising gate

class QuditPlusGate(cirq.SingleQubitGate):
    def __init__(self, dim):
        self.dim = dim
        super(QuditPlusGate, self)
        
    def _qid_shape_(self):
        return (self.dim,)

    def _unitary_(self):
        unitary = np.zeros((self.dim, self.dim))
        for i in range(self.dim):
            unitary[(i + 1) % self.dim, i] = 1
            
        return unitary

    def _circuit_diagram_info_(self, args):
        return '[+1]'
    

In [7]:
#Qudit lowering gate

class QuditMinusGate(cirq.SingleQubitGate):
    def __init__(self, dim):
        self.dim = dim
        super(QuditMinusGate, self)
        
    def _qid_shape_(self):
        return (self.dim,)

    def _unitary_(self):
        unitary = np.zeros((self.dim, self.dim))
        for i in range(self.dim):
            unitary[(i + self.dim - 1) % self.dim, i] = 1
            
        return unitary

    def _circuit_diagram_info_(self, args):
        return '[-1]'

In [8]:
class QuditZGate(cirq.SingleQubitGate):
    def __init__(self, dim):
        self.dim = dim
        super(QuditZGate, self)
        
    def _qid_shape_(self):
        return (self.dim,)

    def _unitary_(self):
        unitary = np.zeros((self.dim, self.dim), dtype = np.complex_)
        w = np.exp(2 * np.pi * 1j / self.dim)
        
        for i in range(self.dim):
            unitary[i, i] = w ** i
            
        return unitary

    def _circuit_diagram_info_(self, args):
        return 'Z'

In [10]:
def seeState(circuit):
    s = cirq.Simulator()
    results = s.simulate(circuit)
    
    print("Circuit: ")
    print(circuit)
    print("\nThe state is:\n {}".format(results.dirac_notation()))
    
def getDirac(circuit):
    s = cirq.Simulator()
    results = s.simulate(circuit)
    
    return results.dirac_notation()

def entangle(q0, q1, circuit, dim):
    circuit.append(QuditHadamard(dim).on(q0))
    circuit.append(QuditCnot(dim).on(q0, q1))
    
def aliceOperation(values, q0, circuit, dim):
    for i in range(values[1]):
        circuit.append(QuditMinusGate(dim).on(q0))
    
    for i in range(values[0]):
        circuit.append(QuditZGate(dim).on(q0))
    
        
def bobOperation(q0, q1, circuit, dim):
    circuit.append(QuditInverseCnot(dim).on(q0, q1))
    circuit.append(QuditInverseHadamard(dim).on(q0))

In [11]:
dim = 4

#Define 2 digit number to be transmitted
values = (3, 2)

#Entangle qutrits
q0 = cirq.LineQid(0, dimension=dim)
q1 = cirq.LineQid(1, dimension=dim)
circuit = cirq.Circuit()
entangle(q0, q1, circuit, dim)
seeState(circuit)


Circuit: 
0 (d=4): ───[H+]───O───
                   │
1 (d=4): ──────────X───

The state is:
 0.5|00⟩ + 0.5|11⟩ + 0.5|22⟩ + 0.5|33⟩


In [12]:
#Alice operations
aliceOperation(values, q0, circuit, dim)
seeState(circuit)

Circuit: 
0 (d=4): ───[H+]───O───[-1]───[-1]───Z───Z───Z───
                   │
1 (d=4): ──────────X─────────────────────────────

The state is:
 0.5|02⟩ - 0.5j|13⟩ - 0.5|20⟩ + 0.5j|31⟩


In [13]:
#Bob operations
bobOperation(q0, q1, circuit, dim)
seeState(circuit)

Circuit: 
0 (d=4): ───[H+]───O───[-1]───[-1]───Z───Z───Z───O───[H-]───
                   │                             │
1 (d=4): ──────────X─────────────────────────────Y──────────

The state is:
 |32⟩


In [14]:
#Test all combinations for dim = 4
dim = 4
for i in range(dim):
    for j in range(dim):
        #The message Alice encodes
        pair = (i, j)
        
        q0 = cirq.LineQid(0, dimension=dim)
        q1 = cirq.LineQid(1, dimension=dim)
        circuit = cirq.Circuit()
        entangle(q0, q1, circuit, dim)
        
        aliceOperation(pair, q0, circuit, dim)
        bobOperation(q0, q1, circuit, dim)
        print("Alice gets {}, Bob measures {}".format(pair, getDirac(circuit)))

Alice gets (0, 0), Bob measures |00⟩
Alice gets (0, 1), Bob measures |01⟩
Alice gets (0, 2), Bob measures |02⟩
Alice gets (0, 3), Bob measures |03⟩
Alice gets (1, 0), Bob measures |10⟩
Alice gets (1, 1), Bob measures |11⟩
Alice gets (1, 2), Bob measures |12⟩
Alice gets (1, 3), Bob measures |13⟩
Alice gets (2, 0), Bob measures |20⟩
Alice gets (2, 1), Bob measures |21⟩
Alice gets (2, 2), Bob measures |22⟩
Alice gets (2, 3), Bob measures |23⟩
Alice gets (3, 0), Bob measures |30⟩
Alice gets (3, 1), Bob measures |31⟩
Alice gets (3, 2), Bob measures |32⟩
Alice gets (3, 3), Bob measures |33⟩


In [15]:
#Test all combinations for dim = 5
dim = 5
for i in range(dim):
    for j in range(dim):
        #The message Alice encodes
        pair = (i, j)
        
        q0 = cirq.LineQid(0, dimension=dim)
        q1 = cirq.LineQid(1, dimension=dim)
        circuit = cirq.Circuit()
        entangle(q0, q1, circuit, dim)
        
        aliceOperation(pair, q0, circuit, dim)
        bobOperation(q0, q1, circuit, dim)
        print("Alice gets {}, Bob measures {}".format(pair, getDirac(circuit)))

Alice gets (0, 0), Bob measures |00⟩
Alice gets (0, 1), Bob measures |01⟩
Alice gets (0, 2), Bob measures |02⟩
Alice gets (0, 3), Bob measures |03⟩
Alice gets (0, 4), Bob measures |04⟩
Alice gets (1, 0), Bob measures |10⟩
Alice gets (1, 1), Bob measures |11⟩
Alice gets (1, 2), Bob measures |12⟩
Alice gets (1, 3), Bob measures |13⟩
Alice gets (1, 4), Bob measures |14⟩
Alice gets (2, 0), Bob measures |20⟩
Alice gets (2, 1), Bob measures |21⟩
Alice gets (2, 2), Bob measures |22⟩
Alice gets (2, 3), Bob measures |23⟩
Alice gets (2, 4), Bob measures |24⟩
Alice gets (3, 0), Bob measures |30⟩
Alice gets (3, 1), Bob measures |31⟩
Alice gets (3, 2), Bob measures |32⟩
Alice gets (3, 3), Bob measures |33⟩
Alice gets (3, 4), Bob measures |34⟩
Alice gets (4, 0), Bob measures |40⟩
Alice gets (4, 1), Bob measures |41⟩
Alice gets (4, 2), Bob measures |42⟩
Alice gets (4, 3), Bob measures |43⟩
Alice gets (4, 4), Bob measures |44⟩


In [16]:
#Test all combinations for dim = 6
dim = 6
for i in range(dim):
    for j in range(dim):
        #The message Alice encodes
        pair = (i, j)
        
        q0 = cirq.LineQid(0, dimension=dim)
        q1 = cirq.LineQid(1, dimension=dim)
        circuit = cirq.Circuit()
        entangle(q0, q1, circuit, dim)
        
        aliceOperation(pair, q0, circuit, dim)
        bobOperation(q0, q1, circuit, dim)
        print("Alice gets {}, Bob measures {}".format(pair, getDirac(circuit)))

Alice gets (0, 0), Bob measures |00⟩
Alice gets (0, 1), Bob measures |01⟩
Alice gets (0, 2), Bob measures |02⟩
Alice gets (0, 3), Bob measures |03⟩
Alice gets (0, 4), Bob measures |04⟩
Alice gets (0, 5), Bob measures |05⟩
Alice gets (1, 0), Bob measures |10⟩
Alice gets (1, 1), Bob measures |11⟩
Alice gets (1, 2), Bob measures |12⟩
Alice gets (1, 3), Bob measures |13⟩
Alice gets (1, 4), Bob measures |14⟩
Alice gets (1, 5), Bob measures |15⟩
Alice gets (2, 0), Bob measures |20⟩
Alice gets (2, 1), Bob measures |21⟩
Alice gets (2, 2), Bob measures |22⟩
Alice gets (2, 3), Bob measures |23⟩
Alice gets (2, 4), Bob measures |24⟩
Alice gets (2, 5), Bob measures |25⟩
Alice gets (3, 0), Bob measures |30⟩
Alice gets (3, 1), Bob measures |31⟩
Alice gets (3, 2), Bob measures |32⟩
Alice gets (3, 3), Bob measures |33⟩
Alice gets (3, 4), Bob measures |34⟩
Alice gets (3, 5), Bob measures |35⟩
Alice gets (4, 0), Bob measures |40⟩
Alice gets (4, 1), Bob measures |41⟩
Alice gets (4, 2), Bob measures |42⟩
A